# Analysis of Antarctic Sea Ice Extension: Annual Maxima and Minima

[<img src="https://colab.research.google.com/img/colab_favicon_256px.png" height="35px" align=CENTER> Open in Google Colab ](https://colab.research.google.com/drive/1WZ1gh8o4G1Whb-xiiEk5ztBmBCBLKAVc)<br>
<sub>(To open link in new tab press Ctrl + click)</sub>

The extent of Antarctic Ice Sheet serves as a critical indicator for understanding climate change dynamics and marine ecosystem shifts in one of the Earth’s most remote regions.

The extent of Antarctic Ice Sheet is usually a remote sensing Earth Observation product.


This notebook processes and presents the annual minimum and maximum extent of the AIS.
The tool uses the following products:

- Copernicus Marine Service
- EMODnet
- ...

All the data is available in the OCEAN:ICE backend data tools.  
The code (CC-BY – DOI(?)) is downloadable and may serve as a starting tool for further re-analysis

The following python libraries are used:
- [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/)
- [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/)
- [requests](https://requests.readthedocs.io/en/latest/)
- [functools](https://docs.python.org/3/library/functools.html)
- [json](https://docs.python.org/3/library/json.html

In [18]:
!pip install ipywidgets==8.1.1 requests==2.31.0 ipyleaflet==0.18.1
%reload_ext autoreload
%autoreload 2

In [19]:
'''This block of code gets a list of geojsons containing informations about the antarctic ice sheet by year.
The function reduceFun creates a dictionary with the data needed for the following cells of code'''
from functools import reduce
import requests
import json

resp = requests.get('https://prod-geoserver.emodnet-physics.eu/geoserver/EMODnet/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=EMODnet%3Aice_edge_sh_annual&outputFormat=application%2Fjson&srsName=epsg:4326')

iceJson = resp.content
iceData = json.loads(iceJson)

def reduceFun(a, c):
  if c['properties']['year'] not in a.keys():
    a[c['properties']['year']] = {'type': "FeatureCollection", 'features':[]}
  a[c['properties']['year']]['features'].append(c)
  
  return a

iceDict = reduce(reduceFun, iceData['features'], {})

In [20]:
'''This function is used to create colored layers, it will be used in the next block of code'''
def colorFun(feature):
  if feature['properties']['minmax'] == 0:
    return {
        'color': 'black',
        'fillColor': 'red'
    }
  else:
    return {
        'color': 'black',
        'fillColor': 'blue'
    }

In [22]:
'''Here the code creates an interactive graphic representation of the antarctic ice sheet by year'''
from ipyleaflet import Map, Marker, basemaps, WMSLayer, projections, GeoJSON, WidgetControl, LegendControl
from ipywidgets import interact, SelectionSlider

years = list(iceDict.keys())

years.sort()

actualYear = years[0]

selection = SelectionSlider(
    options= years,
    value= actualYear,
    description='year',
    orientation='horizontal',
    slider_color='black',
)

m3 = Map(basemap=basemaps.Esri.AntarcticBasemap, crs=projections.EPSG3031.ESRIBasemap, center=(-90, 0), zoom=2)
widget_control = WidgetControl(widget=selection, position='topright')
m3.add_control(widget_control)

legend = LegendControl({"min":"blue", "max":"red"}, name="", position="topleft")
m3.add_control(legend)

EP = dict(
    name='EPSG:3031',
    custom=True,
    proj4def="""+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1
        +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs""",
    bounds=[[-3174450,-2816050],[2867175,2406325]]
)

geo_json = GeoJSON(
    data = iceDict[actualYear],
    style={
        'opacity': 1, 'fillOpacity': 0.5, 'weight': 1
    },
    style_callback=colorFun
)

m3.add_layer(geo_json)

def f(x, year):
  selectedYear = x['owner'].value
  actualYear = selectedYear
  if selectedYear != year:
    year = selectedYear
    m3.remove_layer(geo_json)
    geo_json.data = iceDict[actualYear]
    m3.add(geo_json)

selection.observe(lambda x: f(x, actualYear))

display(m3)

Map(center=[-90, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…